In [1]:
import numpy as np
import pandas as pd

# 3 数据规整化：合并、清理、过滤

## 3.1 入门
#### 3.1.1 函数说明

In [8]:
df1 = pd.DataFrame([[1,2],[3,4]], index=['a', 'b'], columns=['A', 'B'])
df2 = pd.DataFrame([[5,6],[4,8]], index=['b', 'd'], columns=['B', 'C'])
df1

,A,B
a,1,2
b,3,4


In [9]:
df2

,B,C
b,5,6
d,4,8


如果单纯的按照index对齐，不如用concat方法，所以一般不推荐使用left_index, right_index。区别是concat不会对重名的列重命名。

In [10]:
pd.merge(left=df1, right=df2, how="inner", left_index=True, right_index=True)

,A,B_x,B_y,C
b,3,4,5,6


In [11]:
pd.merge(left=df1, right=df2, how="outer", left_index=True, right_index=True)

,A,B_x,B_y,C
a,1.0,2.0,NaN,NaN
b,3.0,4.0,5.0,6.0
d,NaN,NaN,4.0,8.0


In [12]:
pd.concat([df1, df2], join='inner', axis=1)

,A,B,B,C
b,3,4,5,6


#### 3.1.2 on用法

In [14]:
# 对于B列：df1中的'b'行，df2中的'd'行是相同的，其他都不同。相当于找到B列中元素相等的行，合并，并去调重复的列。
pd.merge(left=df1, right=df2, how='inner', on=['B'])

,A,B,C
0,3,4,8


In [16]:
# df1的A列a行，df2的C列b行是相同的，其他都不同。
# 其他列同名会进行重命名。
df1 = pd.DataFrame([[1,2],[3,4]], index=['a', 'b'], columns=['A', 'B'])
df2 = pd.DataFrame([[5,1],[4,8]], index=['b', 'd'], columns=['B', 'C'])
pd.merge(left=df1, right=df2,how='inner',left_on=['A'], right_on=['C'])

,A,B_x,B_y,C
0,1,2,5,1


#### 3.1.3 how 用法

In [17]:
# 保持左侧的DataFrame不变， 用右侧来跟它对齐，对不上的填NaN.
pd.merge(left=df1, right=df2, how='left', on=['B'])

,A,B,C
0,1,2,NaN
1,3,4,8.0


In [18]:
pd.merge(left=df1, right=df2, how='right', on=['B'])

,A,B,C
0,3.0,4,8
1,NaN,5,1


对齐的列存在